In [ ]:
import asyncio
import matplotlib.pyplot as plt
from lsst.ts import salobj, utils
from lsst.ts.idl.enums.MTDome import SubSystemId
%matplotlib widget

# Create the plot
fig, axes = plt.subplots(nrows=1, ncols=2)
ax0, ax1 = axes.flatten()
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = False
hfig = display(fig, display_id=True)
fig.canvas.draw()

# Quantities to plot
time=[]
position=[]
velocity=[]

# Initialize MTDome and set to ENABLED
domain = salobj.Domain()
dome = salobj.Remote(domain=domain, name="MTDome")
await dome.start_task
await dome.cmd_start.set_start(timeout=20, settingsToApply="eie")
await salobj.set_summary_state(remote=dome, state=salobj.State.ENABLED, settingsToApply="eie")

# Utility function to plot the position and velocity against time
async def update():
    start_time = utils.current_tai() - 37
    while True:
        az_status = await dome.tel_azimuth.next(flush=True)
        time.append(az_status.timestamp - start_time)
        position.append(az_status.positionActual)
        velocity.append(az_status.velocityActual)
        ax0.clear()
        ax0.plot(time, position)
        ax1.clear()
        ax1.plot(time, velocity)
        hfig.update(fig)

# Create a task to update the plot frequently
loop = asyncio.get_event_loop()
loop.create_task(update())

In [ ]:
await dome.cmd_moveAz.set_start(position=30, velocity=0.01)

In [ ]:
await dome.cmd_moveAz.set_start(position=40, velocity=0.001)

In [ ]:
sub_system_ids = SubSystemId.AMCS
await dome.cmd_stop.set_start(engageBrakes=False, subSystemIds=sub_system_ids)

In [ ]:
await dome.cmd_moveAz.set_start(position=45, velocity=0.001)

In [ ]:
await dome.cmd_park.set_start()

In [ ]:
await salobj.set_summary_state(remote=dome, state=salobj.State.DISABLED)
await salobj.set_summary_state(remote=dome, state=salobj.State.OFFLINE)
await dome.close()
await domain.close()